# Querying

This notebook demonstrates how to retrieve, query and search data using the Forge.

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

## Retrieval

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
resource = forge.retrieve(jane.id)

In [ ]:
resource == jane

### specific version

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
forge.tag(jane, "v1")

In [ ]:
jane.email = "jane.doe@epfl.ch"

In [ ]:
forge.update(jane)

In [ ]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

In [ ]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [ ]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [ ]:
jane_v1 == jane_v1_tag

In [ ]:
print(jane_v1)

### error handling

In [ ]:
resource = forge.retrieve("123")

In [ ]:
print(resource)

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
association_jane = Resource(type="Dataset", contribution=jane)

In [ ]:
association_john = Resource(type="Dataset", contribution=john)

In [ ]:
associations = [association_jane, association_john]

In [ ]:
forge.register(associations)

`Entity` is a known type by the Model, and the `paths` method will load the data structure for the given type. Refer to the `11 - Modeling.ipynb` notebook to learn about Modeling and Types.

In [ ]:
p = forge.paths("Dataset")

You have autocompletion on `p` and this can be used to build a search.

In [ ]:
resources = forge.search(p.type.id == "Dataset", limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

In [ ]:
forge.as_dataframe(resources)

In [ ]:
forge.as_dataframe(resources, store_metadata=True)

### nested field querying

You have autocompletion on `p` but also on nested properties like `p.agent`.

In [ ]:
resources = forge.search(p.type.id == "Dataset", p.contribution.name == "John Smith", limit=3)

In [ ]:
len(resources)

In [ ]:
print(resources[0])

## Graph traversing

SPARQL is used to traverse the graph. The user can provide simplified queries: no prefixes, no compacted URIs, nor full URIs is required, only the term (property) available in the Model to the user.  Refer to the `11 - Modeling.ipynb` notebook to learn about Templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
association = Resource(type="Dataset", contribution=[jane, john])

In [ ]:
forge.register(association)

In [ ]:
forge.template("Dataset")

In [ ]:
query = """
    SELECT ?x ?name
    WHERE {
        ?x a Dataset ;
           contribution ?contributor .
        ?contributor name ?name .
    }
"""

In [ ]:
resources = forge.sparql(query, limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

### rewritten query display

In [ ]:
forge.sparql(query, debug=True)

## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
distribution = forge.attach("../../data")

In [ ]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [ ]:
forge.register(association)

In [ ]:
forge.download(association, "distribution.contentUrl", "./downloaded/")

In [ ]:
! ls -l ./downloaded/

In [ ]:
# ! rm -R ./downloaded/